# Imports

In [1]:
import pandas as pd
import numpy as np

import re #регулярные выражения
import nltk
from nltk.tokenize import word_tokenize #для маркировки текста
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing.text import Tokenizer #преобразования маркированных слов в числа

2022-08-31 09:53:41.687157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-31 09:53:41.687219: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Смотрим на данные и предворительно обрабатываем их

In [2]:
df = pd.read_csv('./data/poems.csv')
df.head()

,writer,poem,text
0,Лермонтов Михаил Юрьевич,Любовь мертвеца,NaN
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ..."


In [3]:
df.nunique()

writer       48
poem      18317
text      18767
dtype: int64

In [4]:
df.isnull().sum()

writer     0
poem       0
text      14
dtype: int64

In [5]:
# удаляю строки без стихов
df['text'].replace('', None, inplace = True)
df.dropna(subset = ['text'], inplace = True)
df.isnull().sum()

writer    0
poem      0
text      0
dtype: int64

In [6]:
# сохраняю файл в текстовом виде
np.savetxt(r'./data/poems.txt', df['text'].values, fmt='%s')

## Предварительная обработка текстового файла

In [7]:
poems = open('./data/poems.txt').read()

In [8]:
#Функция для очищений текста
def preprocess_text(line):
    #Убираю пунктуацию, числа и специальные символы
    sentance = re.sub('[^а-яА-Я]', ' ', line)

    #Удаление одного символа
    sentance = re.sub(r"\s+[а-яА-Я]\s+", ' ', sentance)

    #Удаление нескольких пробелов
    sentance = re.sub(r"\s+", ' ', sentance)

    return sentance.lower()

In [9]:
poems = preprocess_text(poems)

In [10]:
poems[:500]

'на серебряные шпоры в раздумии гляжу за тебя скакун мой скорый за бока твои дрожу наши предки их не знали гарцуя средь степей толстой плеткой погоняли недоезжаных коней но успехом просвещенья вместо грубой старины введены изобретенья чужеземной стороны наше время кормят холят берегут спинную честь прежде били нынче колют что же выгодней бог весть пилигрим аллах ли там среди пустыни застывших волн воздвиг твердыни притоны ангелам своим иль дивы словом роковым стеной умели так высоко громады скал '

# Преобразование слов в числа
Прежде чем мы сможем преобразовать слова в целые числа, нам нужно маркировать наш текст в отдельные слова. Для этого можно использовать метод **word_tokenize()** из модуля **nltk.tokenize** .

In [11]:
nltk.download('punkt')

poems_words = (word_tokenize(poems))
num_words = len(poems_words)
unique_words = len(set(poems_words))

print(f'Total words: {num_words}')
print(f'Unique words: {unique_words}')

[nltk_data] Downloading package punkt to /home/victor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total words: 3258715
Unique words: 244855


In [12]:
#преобразование маркировнных слов в числа
tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(poems_words)

In [13]:
#доступ к словарю с словами и индексами
vocab_size = len(tokenizer.word_index) + 1
word2index = tokenizer.word_index

In [14]:
#проверка
print(poems_words[500])
print(word2index[poems_words[500]])

страдал
3956


# Изменение формы данных

In [15]:
input_sequence = []
output_words = []
input_seq_length = 100 #значения входной последовательности

for i in range(0, num_words - input_seq_length, 1):
    in_seq = poems_words[i:i + input_seq_length]
    out_seq = poems_words[i + input_seq_length]

    input_sequence.append([word2index[word] for word in in_seq])
    output_words.append(word2index[out_seq])

In [ ]:
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))
X = X / float(vocab_size)

y = to_categorical(output_words)

print("X shape:", X.shape)
print("y shape:", y.shape)

# Обучение модели

In [ ]:
model = Sequential()
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(800, return_sequences=True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

model.complie(loss='categorical_crossentropy', optimizer='adam')